# 01_EDA v0.1: Análisis Exploratorio de Datos (Robusto)

Este notebook realiza un análisis exploratorio exhaustivo de los datos disponibles para el proyecto de detección de fraude financiero. 

**Objetivos de esta versión 0.1**:
1. Cargar y revisar todos los archivos de `data/raw/` (o `data/backup/` si los ligeros no existen en raw).  
2. Normalizar tipos (fechas, montos, flags “Yes/No” → 1/0) y revisar datos faltantes.  
3. Combinar transacciones con etiquetas de forma correcta (según la estructura real de `train_fraud_labels.json`).  
4. Detectar duplicados, outliers y valores inconsistentes.  
5. Explorar:
   - Distribución de montos (`amount`).  
   - Balance de la variable objetivo (`target`).  
   - Fraude por estado y ciudad.  
   - Fraude por hora del día y día de la semana.  
   - Fraude por tipo de tarjeta.  
   - Top MCCs donde hay más fraudes.  
6. Calcular algunas correlaciones básicas y visualizar mapa de calor.  
7. Hacer un pequeño preprocesamiento (generar columna `hour`, flags binarias, parsear fechas).  
8. Guardar el dataset combinado limpio en `data/processed/` para la fase de modelado.

In [25]:
# 2.1 Importar librerías esenciales
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# 2.2 Estilo para gráficos
%matplotlib inline
plt.style.use('default')

# 2.3 Opciones de pandas para visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

# 2.4 Función auxiliar para inspeccionar cualquier DataFrame
def inspeccionar_df(df, nombre):
    """
    Imprime info básica: shape, dtypes, nulos y duplicados de un DataFrame.
    """
    print(f"\n--- Inspección de {nombre} ---")
    print("Dimensiones:", df.shape)
    print("\nTipos de columnas:\n", df.dtypes)
    print("\nValores faltantes por columna:\n", df.isnull().sum().sort_values(ascending=False))
    print(f"\nDuplicados exactos (filas idénticas): {df.duplicated().sum()}")
    if "transaction_id" in df.columns:
        print("Duplicados en 'transaction_id':", df["transaction_id"].duplicated().sum())
    print("-" * 60 + "\n")

# 2.5 Semilla para reproducibilidad (opcional)
import random
random.seed(42)


In [29]:
path_labels = "../data/backup/train_fraud_labels.json"
print("¿Existe el archivo?", os.path.exists(path_labels))


# 3.2.2 Detectar tipo de estructura: JSONL, lista o dict
with open(path_labels, "r") as f:
    first_line = f.readline().strip()
    f.seek(0)  # Volver al inicio del archivo

    try:
        json.loads(first_line)
        is_json_lines = True
    except json.JSONDecodeError:
        is_json_lines = False


# 3.2.3 Cargar el archivo según la estructura detectada
try:
    if is_json_lines:
        df_labels = pd.read_json(path_labels, lines=True)
        print("✅ 3.2.3 Cargado como JSON Lines.")
    else:
        with open(path_labels, "r") as f:
            raw_data = json.load(f)

        if isinstance(raw_data, list):
            df_labels = pd.DataFrame(raw_data)
            print("✅ 3.2.3 Cargado como lista de diccionarios.")
        elif isinstance(raw_data, dict):
            df_labels = pd.DataFrame.from_dict(raw_data, orient="index").reset_index()
            df_labels.columns = ["transaction_id", "target"]  # Ajustar si es necesario
            print("✅ 3.2.3 Cargado como diccionario con IDs.")
        else:
            raise ValueError("❌ 3.2.3 Estructura JSON no reconocida.")

    print("Dimensiones de df_labels:", df_labels.shape)
    display(df_labels.head())

except Exception as e:
    print("❌ 3.2.3 Error durante la carga:", e)

¿Existe el archivo? True
✅ 3.2.3 Cargado como JSON Lines.
Dimensiones de df_labels: (1, 1)


,target
0,"{'10649266': 'No', '23410063': 'No', '9316588'..."


In [36]:
# 3.3.1 Cargar transactions_data.csv
path_transactions = "../data/backup/transactions_data.csv"
df_transacciones = pd.read_csv(path_transactions)
inspeccionar_df(df_transacciones, "df_transacciones")

# 3.3.2 Cargar train_fraud_labels.json 'bruto'
path_labels = "../data/backup/train_fraud_labels.json"
with open(path_labels, "r") as f:
    raw_labels = json.load(f)

# 3.3.3 Analizar estructura de raw_labels antes de convertirlo
print("Tipo de raw_labels:", type(raw_labels))
if isinstance(raw_labels, dict):
    print("Claves del diccionario raíz:", raw_labels.keys())
elif isinstance(raw_labels, list):
    print("Primeros 5 elementos de la lista raw_labels:")
    display(raw_labels[:5])
else:
    print("Estructura inesperada en raw_labels.")



--- Inspección de df_transacciones ---
Dimensiones: (13305915, 12)

Tipos de columnas:
 id                  int64
date               object
client_id           int64
card_id             int64
amount             object
use_chip           object
merchant_id         int64
merchant_city      object
merchant_state     object
zip               float64
mcc                 int64
errors             object
dtype: object

Valores faltantes por columna:
 errors            13094522
zip                1652706
merchant_state     1563700
id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
mcc                      0
dtype: int64

Duplicados exactos (filas idénticas): 0
------------------------------------------------------------

Tipo de raw_labels: <class 'dict'>
Claves del diccionario raíz: dict_keys(['target'])


In [31]:
with open(path_labels, "r") as f:
    raw_labels = json.load(f)
type(raw_labels)


dict

In [ ]:
# 4.1 Procesamiento y combinación de etiquetas con transacciones

# 4.1.1 Extraer lista de targets
target_list = raw_labels["target"]

# 4.1.2 Validar longitud
n = len(target_list)
print(f"Se encontraron {n} etiquetas. Las primeras {n} transacciones serán consideradas como 'entrenamiento'.")

# 4.1.3 Crear conjuntos de entrenamiento y prueba
df_train = df_transacciones.iloc[:n].copy()
df_train["target"] = target_list

df_test = df_transacciones.iloc[n:].copy()
df_test["target"] = np.nan

# 4.1.4 Concatenar para análisis completo
df_full = pd.concat([df_train, df_test], ignore_index=True)

# 5 Validación de combinación
print("✅ Unión de datos completada")
print("Dimensiones de df_full:", df_full.shape)
print("Targets disponibles:")
print(" - 0:", (df_full["target"] == 0).sum())
print(" - 1:", (df_full["target"] == 1).sum())
print(" - NaN (sin etiqueta):", df_full["target"].isna().sum())

inspeccionar_df(df_full, "df_full")
display(df_full.head())


Se encontraron 8914963 etiquetas. Las primeras 8914963 transacciones serán consideradas como 'entrenamiento'.
✅ Unión de datos completada
Dimensiones de df_full: (13305915, 13)
Targets disponibles:
 - 0: 0
 - 1: 0
 - NaN (sin etiqueta): 13305915

--- Inspección de df_full ---
Dimensiones: (13305915, 13)

Tipos de columnas:
 id                  int64
date               object
client_id           int64
card_id             int64
amount             object
use_chip           object
merchant_id         int64
merchant_city      object
merchant_state     object
zip               float64
mcc                 int64
errors             object
target             object
dtype: object

Valores faltantes por columna:
 target            13305915
errors            13094522
zip                1652706
merchant_state     1563700
id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id        

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,target
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.00,5499,NaN,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.00,5311,NaN,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.00,4829,NaN,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.00,4829,NaN,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.00,5813,NaN,NaN


In [40]:
# 6.1.1 Convertir 'date' en df_full
if "date" in df_full.columns:
    df_full["date"] = pd.to_datetime(df_full["date"])
    df_full["hour"] = df_full["date"].dt.hour
    df_full["dayofweek"] = df_full["date"].dt.dayofweek  # 0 = lunes ... 6 = domingo
    df_full["month"] = df_full["date"].dt.month
    print("Columnas temporales añadidas: hour, dayofweek, month")
else:
    print("No existe columna 'date' en df_full.")
    
# Inspección rápida
inspeccionar_df(df_full[["date", "hour", "dayofweek", "month"]], "df_full (columnas temporales)")


Columnas temporales añadidas: hour, dayofweek, month

--- Inspección de df_full (columnas temporales) ---
Dimensiones: (13305915, 4)

Tipos de columnas:
 date         datetime64[ns]
hour                  int32
dayofweek             int32
month                 int32
dtype: object

Valores faltantes por columna:
 date         0
hour         0
dayofweek    0
month        0
dtype: int64

Duplicados exactos (filas idénticas): 9169419
------------------------------------------------------------



In [41]:
# 6.2.1 Inspeccionar df_cards antes de cambios
inspeccionar_df(df_cards, "df_cards (antes de limpieza)")

# Ejemplo de conversión si df_cards tiene:
# 'credit_limit' en formato string como '$1,000.00', 'income' similar, 'debt' similar, 
# 'active_card' en "Yes"/"No", etc.
if "credit_limit" in df_cards.columns:
    # Quitar símbolos de dólar y comas, convertir a float
    df_cards["credit_limit"] = (
        df_cards["credit_limit"]
        .replace({'\$': '', ',': ''}, regex=True)
        .astype(float)
    )
if "income" in df_cards.columns:
    df_cards["income"] = (
        df_cards["income"]
        .replace({'\$': '', ',': ''}, regex=True)
        .astype(float)
    )
if "debt" in df_cards.columns:
    df_cards["debt"] = (
        df_cards["debt"]
        .replace({'\$': '', ',': ''}, regex=True)
        .astype(float)
    )
# Convertir flags "Yes"/"No" a 1/0 si existieran columnas así:
for col in df_cards.columns:
    if df_cards[col].dtype == object and set(df_cards[col].dropna().unique()) == {"Yes", "No"}:
        df_cards[col] = df_cards[col].map({"Yes": 1, "No": 0})

# 6.2.2 Inspeccionar df_cards tras limpieza
inspeccionar_df(df_cards, "df_cards (después de limpieza)")



--- Inspección de df_cards (antes de limpieza) ---
Dimensiones: (6146, 13)

Tipos de columnas:
 id                        int64
client_id                 int64
card_brand               object
card_type                object
card_number               int64
expires                  object
cvv                       int64
has_chip                 object
num_cards_issued          int64
credit_limit             object
acct_open_date           object
year_pin_last_changed     int64
card_on_dark_web         object
dtype: object

Valores faltantes por columna:
 id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64

Duplicados exactos (filas idénticas): 0
-----------------------------------

<>:11: SyntaxWarning: invalid escape sequence '\$'
<>:17: SyntaxWarning: invalid escape sequence '\$'
<>:23: SyntaxWarning: invalid escape sequence '\$'
<>:11: SyntaxWarning: invalid escape sequence '\$'
<>:17: SyntaxWarning: invalid escape sequence '\$'
<>:23: SyntaxWarning: invalid escape sequence '\$'
C:\Users\juann\AppData\Local\Temp\ipykernel_19340\2132327296.py:11: SyntaxWarning: invalid escape sequence '\$'
  .replace({'\$': '', ',': ''}, regex=True)
C:\Users\juann\AppData\Local\Temp\ipykernel_19340\2132327296.py:17: SyntaxWarning: invalid escape sequence '\$'
  .replace({'\$': '', ',': ''}, regex=True)
C:\Users\juann\AppData\Local\Temp\ipykernel_19340\2132327296.py:23: SyntaxWarning: invalid escape sequence '\$'
  .replace({'\$': '', ',': ''}, regex=True)


In [42]:
# 6.3.1 Inspeccionar df_users antes de cambios
inspeccionar_df(df_users, "df_users (antes de limpieza)")

# Si df_users tiene campos 'gender' (M/F), 'marital_status' (Yes/No, etc.), 
# o montos similares a df_cards, convertirlos:
if "gender" in df_users.columns:
    df_users["gender"] = df_users["gender"].map({"M": 1, "F": 0})  # ejemplo
# Convertir flags "Yes"/"No" a 1/0
for col in df_users.columns:
    if df_users[col].dtype == object and set(df_users[col].dropna().unique()) == {"Yes", "No"}:
        df_users[col] = df_users[col].map({"Yes": 1, "No": 0})

# 6.3.2 Inspeccionar df_users tras limpieza
inspeccionar_df(df_users, "df_users (después de limpieza)")



--- Inspección de df_users (antes de limpieza) ---
Dimensiones: (2000, 14)

Tipos de columnas:
 id                     int64
current_age            int64
retirement_age         int64
birth_year             int64
birth_month            int64
gender                object
address               object
latitude             float64
longitude            float64
per_capita_income     object
yearly_income         object
total_debt            object
credit_score           int64
num_credit_cards       int64
dtype: object

Valores faltantes por columna:
 id                   0
current_age          0
retirement_age       0
birth_year           0
birth_month          0
gender               0
address              0
latitude             0
longitude            0
per_capita_income    0
yearly_income        0
total_debt           0
credit_score         0
num_credit_cards     0
dtype: int64

Duplicados exactos (filas idénticas): 0
------------------------------------------------------------


--- Inspecc

In [43]:
# 7.1 Verificar si existen las keys en df_cards y df_users
print("card_id en df_full?", "card_id" in df_full.columns)
print("card_id en df_cards?", "card_id" in df_cards.columns)
print("client_id en df_full?", "client_id" in df_full.columns)
print("client_id en df_users?", "client_id" in df_users.columns)

# 7.2 Si existe, hacer merge para anexar columnas de df_cards
if "card_id" in df_full.columns and "card_id" in df_cards.columns:
    columnas_cards = [c for c in df_cards.columns if c != "card_id"]
    df_full = pd.merge(
        df_full,
        df_cards[["card_id"] + columnas_cards],
        how="left",
        on="card_id",
        suffixes=("", "_card")
    )
    print("Dimensiones de df_full tras merge con df_cards:", df_full.shape)

# 7.3 Si existe, anexar columnas de df_users
if "client_id" in df_full.columns and "client_id" in df_users.columns:
    columnas_users = [c for c in df_users.columns if c != "client_id"]
    df_full = pd.merge(
        df_full,
        df_users[["client_id"] + columnas_users],
        how="left",
        on="client_id",
        suffixes=("", "_user")
    )
    print("Dimensiones de df_full tras merge con df_users:", df_full.shape)

# 7.4 Inspeccionar dtypes y nulos tras merges adicionales
inspeccionar_df(df_full, "df_full (después de merges con cards + users)")


card_id en df_full? True
card_id en df_cards? False
client_id en df_full? True
client_id en df_users? False

--- Inspección de df_full (después de merges con cards + users) ---
Dimensiones: (13305915, 16)

Tipos de columnas:
 id                         int64
date              datetime64[ns]
client_id                  int64
card_id                    int64
amount                    object
use_chip                  object
merchant_id                int64
merchant_city             object
merchant_state            object
zip                      float64
mcc                        int64
errors                    object
target                    object
hour                       int32
dayofweek                  int32
month                      int32
dtype: object

Valores faltantes por columna:
 target            13305915
errors            13094522
zip                1652706
merchant_state     1563700
id                       0
date                     0
client_id                0
card_id   

In [44]:
# 8.1.1 Columnas numéricas en df_full
numeric_cols = df_full.select_dtypes(include=["int64", "float64"]).columns.tolist()
print("Columnas numéricas:", numeric_cols)

# 8.1.2 Statistic describe transpuestas
display(df_full[numeric_cols].describe().T)

# 8.1.3 Conteo de duplicados exactos
print("Duplicados exactos en df_full:", df_full.duplicated().sum())


Columnas numéricas: ['id', 'client_id', 'card_id', 'merchant_id', 'zip', 'mcc']


,count,mean,std,min,25%,50%,75%,max
id,13305915.00,15584024.57,4704498.65,7475327.00,11506044.50,15570866.00,19653605.50,23761874.00
client_id,13305915.00,1026.81,581.64,0.00,519.00,1070.00,1531.00,1998.00
card_id,13305915.00,3475.27,1674.36,0.00,2413.00,3584.00,4901.00,6144.00
merchant_id,13305915.00,47723.76,25815.34,1.00,25887.00,45926.00,67570.00,100342.00
zip,11653209.00,51327.82,29404.23,1001.00,28602.00,47670.00,77901.00,99928.00
mcc,13305915.00,5565.44,875.70,1711.00,5300.00,5499.00,5812.00,9402.00


Duplicados exactos en df_full: 0
